# GALSEDATLAS AGN Composites and Applications to the Lyman Break Technique 
This code is intended to test the composite models against against the GALSEDATLAS AGN composites. The code is also intended to test the Lyman Break technique on the GALSEDATLAS AGN composites.